In [1]:
import openpyxl
import pandas as pd
import os
import datetime
import copy
#from IPython.display import display

In [2]:
excelname= "Aroon Oscillator_test"
sheetname = "Aroon Oscillator"

wb = openpyxl.load_workbook(str(excelname)+".xlsx")
ws = wb[str(sheetname)]

dataset = []

i = 1
while(ws.cell(row=i, column=1).value!=None):
    row = []
    row.append(ws.cell(row=i, column=1).value)
    row.append(ws.cell(row=i, column=2).value)
    row.append(ws.cell(row=i, column=3).value)
    row.append(ws.cell(row=i, column=4).value)
    row.append(ws.cell(row=i, column=5).value)
    
    dataset.append(row)
    i = i + 1
pd.DataFrame(dataset).head()

,0,1,2,3,4
0,date,open,high,low,close
1,2003/06/30,37.1,37.4,36.92,37.08
2,2003/07/01,37.09,38.1,37.09,38.05
3,2003/07/02,38.17,38.82,38.1,38.69
4,2003/07/03,40.6,40.6,38.81,39


In [3]:
trade = copy.deepcopy(dataset)
from datetime import datetime
for i in range(0, len(dataset)):
    for j in range(5, 16):
        if (i != 0):
            trade[i][0] = datetime.strptime(dataset[i][0], '%Y/%m/%d').date()
        trade[i].append(0)

#trade[0][1] = "open"
#trade[0][2] = "high"
#trade[0][3] = "low"
#trade[0][4] = "close"
trade[0][5] = "Aroon-Up"
trade[0][6] = "Aroon-Down"
trade[0][7] = "Aroon Oscillator"
trade[0][8] = "buy"
trade[0][9] = "hold"
trade[0][10] = "sell"
trade[0][11] = "number"
trade[0][12] = "value"
trade[0][13] = "period_return"
trade[0][14] = "holding days"
trade[0][15] = "annual return"

pd.DataFrame(trade).head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,date,open,high,low,close,Aroon-Up,Aroon-Down,Aroon Oscillator,buy,hold,sell,number,value,period_return,holding days,annual return
1,2003-06-30,37.1,37.4,36.92,37.08,0,0,0,0,0,0,0,0,0,0,0
2,2003-07-01,37.09,38.1,37.09,38.05,0,0,0,0,0,0,0,0,0,0,0
3,2003-07-02,38.17,38.82,38.1,38.69,0,0,0,0,0,0,0,0,0,0,0
4,2003-07-03,40.6,40.6,38.81,39,0,0,0,0,0,0,0,0,0,0,0


In [4]:
cycle = 25  #9天
cycle_index = 25

while(cycle_index < len(trade)):
    low = []  #每次清空list
    high = []
    for i in range(cycle, 0, -1):
        low.append(trade[cycle_index - i + 1][3])   #填充9天最低價
        high.append(trade[cycle_index - i + 1][2])  #填充9天最高價
        
    #計算當天K值時必須前一天已經有參考值所以KD值計算放在這裡執行
    if(cycle_index < len(trade)): #因為會跨到隔天所以要先檢查會不會超出LIST範圍 
        AroonUp = 100 * (cycle - (cycle-(high.index(max(high))+1))) / 25 #100 x (25 - Days Since 25-day High)/25
        AroonDown = 100 * (cycle - (cycle-(low.index(min(low))+1))) / 25 #100 x (25 - Days Since 25-day Low)/25
               
#         if(cycle_index == 25):
#             print(cycle)
#             print(high.index(max(high))+1)
#             print(cycle-(high.index(max(high))+1))
#             print(cycle - (cycle-(high.index(max(high))+1)))
        
        trade[cycle_index][5] = AroonUp
        trade[cycle_index][6] = AroonDown
        trade[cycle_index][7] = AroonUp - AroonDown
        
    cycle_index = cycle_index + 1
pd.DataFrame(trade).head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,date,open,high,low,close,Aroon-Up,Aroon-Down,Aroon Oscillator,buy,hold,sell,number,value,period_return,holding days,annual return
1,2003-06-30,37.1,37.4,36.92,37.08,0,0,0,0,0,0,0,0,0,0,0
2,2003-07-01,37.09,38.1,37.09,38.05,0,0,0,0,0,0,0,0,0,0,0
3,2003-07-02,38.17,38.82,38.1,38.69,0,0,0,0,0,0,0,0,0,0,0
4,2003-07-03,40.6,40.6,38.81,39,0,0,0,0,0,0,0,0,0,0,0


In [5]:
for i in range(cycle, len(trade)):
    if(trade[i][7] > 0)and(trade[i-1][9]==0):
        trade[i][8] = 1
        trade[i][9] = 1
        trade[i][10] = 0
        
    elif(trade[i][7] < 0)and(trade[i-1][9]==1):
        trade[i][8] = 0
        trade[i][9] = 0
        trade[i][10] = 1
        
    elif(trade[i-1][9]>0):
        trade[i][8]  = 0
        trade[i][9]  = 1
        trade[i][10] = 0

pd.DataFrame(trade).head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,date,open,high,low,close,Aroon-Up,Aroon-Down,Aroon Oscillator,buy,hold,sell,number,value,period_return,holding days,annual return
1,2003-06-30,37.1,37.4,36.92,37.08,0,0,0,0,0,0,0,0,0,0,0
2,2003-07-01,37.09,38.1,37.09,38.05,0,0,0,0,0,0,0,0,0,0,0
3,2003-07-02,38.17,38.82,38.1,38.69,0,0,0,0,0,0,0,0,0,0,0
4,2003-07-03,40.6,40.6,38.81,39,0,0,0,0,0,0,0,0,0,0,0


In [19]:
initial = 10000
for i in range(cycle, len(trade)):
    if(trade[i][8] == 1)and(trade[i][9]==1)and(trade[i][10] == 0):
        first_day = trade[i][0]
        trade[i][11] = initial/trade[i][1] #數量
        trade[i][12] = trade[i][4] * trade[i][11] #現值
        trade[i][13] = round((trade[i][4] * trade[i][11] - initial)/initial, 4) #期間報酬
        trade[i][14] = (trade[i][0] - first_day).days + 1 #持有天數
        trade[i][15] = (1+trade[i][13])**(365/trade[i][14])-1
        
    elif(trade[i][8] == 0)and(trade[i][9]==0)and(trade[i][10] == 1):
        trade[i][11] = 0
        trade[i][12] = trade[i-1][11] * trade[i][4]
        trade[i][13] = round((trade[i-1][11] * trade[i][4] - initial)/initial, 4)
        trade[i][14] = (trade[i][0] - first_day).days + 1#持有天數
        trade[i][15] = (1+trade[i][13])**(365/trade[i][14])-1
        
    elif(trade[i][8] == 0)and(trade[i][9]==1)and(trade[i][10] == 0):
        trade[i][11] = trade[i-1][11]
        trade[i][12] = trade[i][1] * trade[i][6]
        trade[i][13] = round((trade[i][4] * trade[i][11] - initial)/initial, 4)
        trade[i][14] = (trade[i][0] - first_day).days + 1 #持有天數
        trade[i][15] = (1+trade[i][13])**(365/trade[i][14])-1
        
    else:
        trade[i][14] = (trade[i][0] - trade[i][0]).days
        
pd.DataFrame(trade).tail(50)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
3378,2017-02-14,73.4,73.5,73.1,73.2,68,84,-16,0,0,0,0,0,0,0,0
3379,2017-02-15,73.3,73.9,73.2,73.9,100,80,20,1,1,0,136.426,10081.9,0.0082,1,18.7035
3380,2017-02-16,74,74.2,73.35,73.45,100,76,24,0,1,0,136.426,5624,0.002,2,0.439989
3381,2017-02-17,73.65,73.7,73.35,73.35,96,72,24,0,1,0,136.426,5302.8,0.0007,3,0.0888661
3382,2017-02-18,73.3,73.55,73.2,73.5,92,68,24,0,1,0,136.426,4984.4,0.0027,4,0.278955
3383,2017-02-20,73.55,73.65,73.1,73.1,88,64,24,0,1,0,136.426,4707.2,-0.0027,6,-0.151659
3384,2017-02-21,73.2,73.4,73.05,73.25,84,60,24,0,1,0,136.426,4392,-0.0007,7,-0.0358542
3385,2017-02-22,73.6,73.7,73.15,73.3,80,56,24,0,1,0,136.426,4121.6,-0,8,0
3386,2017-02-23,73.25,73.55,73.1,73.1,76,52,24,0,1,0,136.426,3809,-0.0027,9,-0.103851
3387,2017-02-24,73.05,73.3,72.85,73.15,72,48,24,0,1,0,136.426,3506.4,-0.002,10,-0.0704671


In [7]:
def hyper(dataset, cycle):
    trade = copy.deepcopy(dataset)
    from datetime import datetime
    for i in range(0, len(dataset)):
        for j in range(5, 16):
            if (i != 0):
                trade[i][0] = datetime.strptime(dataset[i][0], '%Y/%m/%d').date()
            trade[i].append(0)

    #trade[0][1] = "open"
    #trade[0][2] = "high"
    #trade[0][3] = "low"
    #trade[0][4] = "close"
    trade[0][5] = "Aroon-Up"
    trade[0][6] = "Aroon-Down"
    trade[0][7] = "Aroon Oscillator"
    trade[0][8] = "buy"
    trade[0][9] = "hold"
    trade[0][10] = "sell"
    trade[0][11] = "number"
    trade[0][12] = "value"
    trade[0][13] = "period_return"
    trade[0][14] = "holding days"
    trade[0][15] = "annual return"


#     cycle = 25  #9天
    cycle_index = cycle

    while(cycle_index < len(trade)):
        low = []  #每次清空list
        high = []
        for i in range(cycle, 0, -1):
            low.append(trade[cycle_index - i + 1][3])   #填充9天最低價
            high.append(trade[cycle_index - i + 1][2])  #填充9天最高價

        #計算當天K值時必須前一天已經有參考值所以KD值計算放在這裡執行
        if(cycle_index < len(trade)): #因為會跨到隔天所以要先檢查會不會超出LIST範圍 
            AroonUp = 100 * (cycle - (cycle-(high.index(max(high))+1))) / cycle #100 x (25 - Days Since 25-day High)/25
            AroonDown = 100 * (cycle - (cycle-(low.index(min(low))+1))) / 25 #100 x (25 - Days Since 25-day Low)/25

    #         if(cycle_index == 25):
    #             print(cycle)
    #             print(high.index(max(high))+1)
    #             print(cycle-(high.index(max(high))+1))
    #             print(cycle - (cycle-(high.index(max(high))+1)))

            trade[cycle_index][5] = AroonUp
            trade[cycle_index][6] = AroonDown
            trade[cycle_index][7] = AroonUp - AroonDown

        cycle_index = cycle_index + 1

    for i in range(cycle, len(trade)):
        if(trade[i][7] > 0)and(trade[i-1][9]==0):
            trade[i][8] = 1
            trade[i][9] = 1
            trade[i][10] = 0

        elif(trade[i][7] < 0)and(trade[i-1][9]==1):
            trade[i][8] = 0
            trade[i][9] = 0
            trade[i][10] = 1

        elif(trade[i-1][9]>0):
            trade[i][8]  = 0
            trade[i][9]  = 1
            trade[i][10] = 0

    initial = 10000
    for i in range(cycle, len(trade)):
        if(trade[i][8] == 1)and(trade[i][9]==1)and(trade[i][10] == 0):
            first_day = trade[i][0]
            trade[i][11] = initial/trade[i][1]
            trade[i][12] = trade[i][4] * trade[i][11]
            trade[i][13] = round((trade[i][4] * trade[i][11] - initial)/initial, 4)   
            trade[i][14] = (trade[i][0] - first_day).days + 1 #持有天數
            trade[i][15] = (1+trade[i][13])**(365/trade[i][14])-1

        elif(trade[i][8] == 0)and(trade[i][9]==0)and(trade[i][10] == 1):
            trade[i][11] = 0
            trade[i][12] = trade[i-1][11] * trade[i][4]
            trade[i][13] = round((trade[i-1][11] * trade[i][4] - initial)/initial, 4)
            trade[i][14] = (trade[i][0] - first_day).days + 1#持有天數
            trade[i][15] = (1+trade[i][13])**(365/trade[i][14])-1

        elif(trade[i][8] == 0)and(trade[i][9]==1)and(trade[i][10] == 0):
            trade[i][11] = trade[i-1][6]
            trade[i][12] = trade[i][1] * trade[i][6]
            trade[i][13] = round((trade[i][4] * trade[i][11] - initial)/initial, 4)
            trade[i][14] = (trade[i][0] - first_day).days + 1 #持有天數
            trade[i][15] = (1+trade[i][13])**(365/trade[i][14])-1

        else:
            trade[i][14] = (trade[i][0] - trade[i][0]).days
            
    annual_return = 1
    trading_counts = 0

    for i in range(1, len(trade)):
        if(trade[i][10]==1):
            annual_return = annual_return * (1+trade[i][13])
            trading_counts = trading_counts + 1
    #     print(annual_return)
    #     print(trading_counts)
    final_return = annual_return**(1/trading_counts)-1
    #     print(final_return)
    return final_return

In [14]:
hyper(dataset, 30)

-0.9651352624769644

In [20]:
for i in range(25, 50, 2):
    print("Thin final annual return of Aroon Oscillator" + str(i)+" is:" + str(hyper(dataset, i)))

Thin final annual return of Aroon Oscillator25 is:-0.9600391578826253
Thin final annual return of Aroon Oscillator27 is:-0.9621327022029132
Thin final annual return of Aroon Oscillator29 is:-0.9589007314510958
Thin final annual return of Aroon Oscillator31 is:-0.9660098740224252
Thin final annual return of Aroon Oscillator33 is:-0.9626600072943456
Thin final annual return of Aroon Oscillator35 is:-0.9619750154045241
Thin final annual return of Aroon Oscillator37 is:-0.9570886008276869
Thin final annual return of Aroon Oscillator39 is:-0.9588216650692196
Thin final annual return of Aroon Oscillator41 is:-0.9573590634276803
Thin final annual return of Aroon Oscillator43 is:-0.9633461375490349
Thin final annual return of Aroon Oscillator45 is:-0.9661264964241025
Thin final annual return of Aroon Oscillator47 is:-0.9639022660495202
Thin final annual return of Aroon Oscillator49 is:-0.9630757849560374
